<!-- trunk-ignore-all(markdownlint/MD001) -->
<!-- trunk-ignore-all(markdownlint/MD024) -->


# Installation of the environment



First, we are setting up our environment, by installing HOOMD-blue and installing the dlext plugin for it. The first step is to update cmake to a version that is compatible with our installation process. In this case, we are using a newer version of CMake that is required by the HOOMD-dlext plugin.
This is special to the underlying (older) Ubuntu system of this Colab (18.04) and should not be required for newer installations.



# Update CMake


In [ ]:
%%bash

wget -O - https://apt.kitware.com/keys/kitware-archive-latest.asc 2> /dev/null | gpg --dearmor - | tee /usr/share/keyrings/kitware-archive-keyring.gpg > /dev/null
echo 'deb [signed-by=/usr/share/keyrings/kitware-archive-keyring.gpg] https://apt.kitware.com/ubuntu/ bionic main' | tee /etc/apt/sources.list.d/kitware.list > /dev/null
apt-get -qq update
apt-get -qq remove cmake > /dev/null
apt-get -qq install cmake > /dev/null

# The following lines are specific to Colab installation environment
[[ -f /usr/local/bin/cmake && ! -f /usr/local/bin/cmake-default ]] && mv /usr/local/bin/cmake /usr/local/bin/cmake-default
ln -sf /usr/bin/cmake /usr/local/bin/cmake


##### \_\_\_

We set up a directory that we will use as an installation prefix (this can be different depending on where you want to install everything).


In [ ]:
import os
import sys

ver = sys.version_info

os.environ["PYSAGES_ENV"] = "/env/pysages"
os.environ["PREFIX"] = (
    "/env/pysages/lib/python" + str(ver.major) + "." + str(ver.minor) + "/site-packages"
)

In [ ]:
!mkdir -p $PREFIX


## HOOMD-blue

With this dependency updated, we can install HOOMD-blue.
The following cell clones and compiles HOOMD-blue manually.
We also make use of disabling components of the HOOMD-blue installation to save some installation time.

**This may take some time, be mindful of not inadvertently re-running the cell.**



# Build and install HOOMD-blue


In [ ]:
%%bash

# Get HOOMD-blue source code
rm -rf hoomd-blue
git clone https://github.com/glotzerlab/hoomd-blue.git &>/dev/null
cd hoomd-blue
git checkout v2.9.7 &> /dev/null

# Compile and install
BUILD_PATH=/tmp/build/hoomd
rm -rf $BUILD_PATH
cmake -S . -B $BUILD_PATH \
    -DCMAKE_INSTALL_PREFIX=$PREFIX \
    -DENABLE_CUDA=ON \
    -DBUILD_CGCMM=OFF \
    -DBUILD_DEM=OFF \
    -DBUILD_DEPRECATED=OFF \
    -DBUILD_HPMC=OFF \
    -DBUILD_METAL=OFF \
    -DBUILD_MPCD=OFF \
    -DBUILD_TESTING=OFF &> /dev/null

cmake --build $BUILD_PATH --target install -j8 &> /dev/null
# Or alternately to the line above
# cd /tmp/build/hoomd
# make install -j8 &> /dev/null


##### \_\_\_

After the successful installation of HOOMD-blue, we make the installation visible to the python system. We installed HOOMD-blue into a custom location that is not checked by python for software out of the box.
In this Colab, this happens in two steps.

First, we extend the environment variable `PYTHONPATH`. This helps newly started python environments to find the HOOMD-blue installation. This is important for the installation of the plugin later. In none-notebook environments, this is the only step necessary.


In [ ]:
import os

os.environ["PYTHONPATH"] = os.environ["PREFIX"] + ":" + os.environ["PYTHONPATH"]


##### \_\_\_

Because the notebook environment has already a running python we need to let this one know about the new package location. We achieve this by appending the `sys.path` with the location of our package.


In [ ]:
import os
import sys

sys.path.append(os.environ["PREFIX"])


## HOOMD-dlext plugin

Now we can install the `dlext` plugin for HOOMD-blue. So we clone the hoomd-dlext repository and install the package via `cmake` as well.
This cell is significantly faster than the HOOMD-blue installation.



##### Build and install HOOMD-dlext plugin


In [ ]:
%%bash

# Get the plugin
rm -rf hoomd-dlext
git clone https://github.com/SSAGESLabs/hoomd-dlext.git &> /dev/null
cd hoomd-dlext

# Build and install
BUILD_PATH=/tmp/build/hoomd-dlext
rm -rf $BUILD_PATH
cmake -S . -B $BUILD_PATH &> /dev/null
cmake --build $BUILD_PATH --target install > /dev/null


##### \_\_\_

This concludes the installation of the HOOMD-blue and its Plugin for PySAGES. We quickly test the installation and, if successful, we proceed to also build and install OpenMM.


In [ ]:
import hoomd
import hoomd.dlext


## OpenMM

Having previously set up the environment variables for the HOOMD-blue installation, we can now just simply install some required dependencies and build and install OpenMM.



##### \_\_\_

Installing dependencies will be different depending on your operating system and python environment.


In [ ]:
%%bash

apt-get -qq install doxygen swig > /dev/null
python -m pip install -q --upgrade pip setuptools wheel Cython &> /dev/null


##### Build and install OpenMM


In [ ]:
%%bash

# Get OpenMM source code
rm -rf openmm
git clone https://github.com/openmm/openmm.git &> /dev/null
cd openmm && git checkout 7.6.0 &> /dev/null

# Compile and install
BUILD_PATH=/tmp/build/openmm
rm -rf $BUILD_PATH
cmake -S . -B $BUILD_PATH \
    -DCMAKE_INSTALL_PREFIX=$PYSAGES_ENV \
    -DBUILD_TESTING=OFF \
    -Wno-dev &> /dev/null

cmake --build $BUILD_PATH -j8 &> /dev/null
cmake --install $BUILD_PATH &> /dev/null

# Install python package
export OPENMM_INCLUDE_PATH=$PYSAGES_ENV/include
export OPENMM_LIB_PATH=$PYSAGES_ENV/lib
cd $BUILD_PATH/python
pip install --target $PREFIX . &> /dev/null


## OpenMM-dlext plugin

Similarly we build and install the corresponding `openmm_dlext` plugin.



##### Build and install openmm-dlext


In [ ]:
%%bash

# Get the plugin
rm -rf openmm-dlext
git clone https://github.com/SSAGESLabs/openmm-dlext.git &> /dev/null
cd openmm-dlext

# Build and install
BUILD_PATH=/tmp/build/openmm-dlext
rm -rf $BUILD_PATH
cmake -S . -B $BUILD_PATH -Wno-dev &> /dev/null
cmake --build $BUILD_PATH --target install > /dev/null


##### \_\_\_

Again, we test the installation and, if successful, we proceed to copy the environment to Google Drive to avoid building everything again in the future.


In [ ]:
import openmm
import openmm_dlext


## Upload environment to Google Drive



**This step can only be successfully executed by PySAGES maintainers.**

These steps are not necessary to understand the setup of the environment. If you want to build your own environment, modify the lines such that it uploads to your own Google Drive.

We upload the data to the shared Google Drive. First, we mount our Google Drive file system to a local directory.


In [ ]:
from google.colab import drive

drive.mount("/content/mnt")

Mounted at /content/mnt



We then compress the environment into a zip file and copy it to a folder within Google Drive. Here we are choosing an existing Shared Drive, but if you were to do this you should choose a folder you have access to and write permissions.


In [ ]:
%env PYSAGES_SHARED_ENV=/content/mnt/Shareddrives/pysages-env

env: PYSAGES_SHARED_ENV=/content/mnt/Shareddrives/pysages-env


In [ ]:
%%bash

cd $PYSAGES_ENV
zip -qr pysages-env.zip .
cp -f pysages-env.zip $PYSAGES_SHARED_ENV
rm -f pysages-env.zip